In [ ]:
import pandas as pd
import numpy as np
import os
from pathlib import Path

import datetime
from datetime import timedelta
# !pip install holidays
import holidays

os.chdir("/content/drive/MyDrive/3. Grad School/LG Aimers")

In [ ]:
# 데이터 불러오기
data = pd.read_csv("DATA/train/train.csv")

#### 파생변수 생성

In [ ]:
# 월(month) -> 계절 매핑 딕셔너리
month_to_season = {
    1: "Winter", 2: "Winter", 12: "Winter",
    3: "Spring", 4: "Spring", 5: "Spring",
    6: "Summer", 7: "Summer", 8: "Summer",
    9: "Autumn", 10: "Autumn", 11: "Autumn"}

In [ ]:
# 월별 가중치 매핑
# monthly_weights = {
#     1: 1.5, 2: 1.5, 3: 0.5,
#     4: 0.8, 5: 0.8, 6: 0.8,
#     7: 1.2, 8: 1.2, 9: 0.8,
#     10: 0.8, 11: 0.8, 12: 1.2}

monthly_weights = {
    1: 2.2, 2: 1.8, 3: 0.3,
    4: 1.01, 5: 0.7, 6: 0.8,
    7: 0.5, 8: 0.5, 9: 0.8,
    10: 1.55, 11: 1.03, 12: 1.4}

In [ ]:
# 요일별 가중치 매핑
# weekly_weights = {
#     "Monday": 0.8, "Tuesday": 0.8, "Wednesday": 0.8,
#     "Thursday": 0.8, "Friday": 1.2, "Saturday": 1.7,
#     "Sunday": 1.5}

weekly_weights = {
    "Monday": 0.78, "Tuesday": 0.85, "Wednesday": 0.81,
    "Thursday": 9.9, "Friday": 1.2, "Saturday": 1.53,
    "Sunday": 1.3}

In [ ]:
class Make_Variables():
        def __init__(self, data = None, date = None, predict = 7, month_to_season = None, monthly_weights = None, weekly_weights = None):
            self.data = data
            self.date = date
            self.predict = predict
            self.month_to_season = month_to_season
            self.monthly_weights = monthly_weights
            self.weekly_weights = weekly_weights

        def update_kor_holidays(self):
            """국경일 추가"""
            kor_holidays = holidays.KR(years = [2023, 2024, 2025])
            kor_holidays.update({
                # datetime.date(2023,2,14) : "Valentine's Day",
                # datetime.date(2023,3,14) : "White Day",
                # datetime.date(2023,11,11) : "Pepero Day",
                # datetime.date(2024,2,14) : "Valentine's Day",
                # datetime.date(2024,3,14) : "White Day",
                # datetime.date(2024,11,11) : "Pepero Day",
                # datetime.date(2025,2,14) : "Valentine's Day",
                # datetime.date(2025,3,14) : "White Day",
                # datetime.date(2025,11,11) : "Pepero Day",

                datetime.date(2024,10,1) : "Temporary Holiday", # 국군의 날 임시공휴일
                datetime.date(2025,1,27) : "Temporary Holiday", # 설날 임시공휴일
                datetime.date(2025,3,3) : "Temporary Holiday", # 삼일절 대체공휴일
                datetime.date(2025, 5, 29) : "Election Period",
                datetime.date(2025, 5, 30) : "Election Period",
                datetime.date(2025, 6, 3) : "Presidential Election Day"})
            return kor_holidays

        def check_holidays(self, date, kor_holidays) -> int:
            """날짜 받아서 공휴일/주말 여부 출력"""
            # date = pd.Timestamp(date)
            if isinstance(date, pd.Series):
                check_holiday = date.dt.date.isin(kor_holidays)
                check_weekend = date.dt.weekday >= 5
            else:
                check_holiday = date.date() in kor_holidays
                check_weekend = date.weekday() >= 5
            is_holiday = (check_holiday | check_weekend)
            return is_holiday

        def get_sandwich_score(self, data, is_holiday_col) -> pd.DataFrame:
            """데이터프레임 기준으로 샌드위치 점수 계산"""
            data = data.reset_index(drop = True)
            data['is_sandwich'] = 0
            is_holiday = data[is_holiday_col].astype(int)
            for idx in range(len(data)):
                if idx == 0 or idx == len(data) - 1: # 첫날, 마지막 날
                    continue

                # 앞/뒤 하루씩 봤을 때 모두 휴일 -> 5점
                if (is_holiday.iloc[idx - 1] == 1) and (is_holiday.iloc[idx + 1] == 1): # 하루 전이랑 다음 날이 공휴일이면
                    data.iloc[idx, data.columns.get_loc('is_sandwich')] = 5

                # 앞/뒤 이틀씩 봤을 때 휴일 3일 -> 3점, 2일 -> 2점
                elif idx > 1 and idx < len(is_holiday) - 2: # 셋째날, 마지막에서 세 번째 날
                    start_idx = idx - 2
                    end_idx = idx + 2
                    nearby_holidays = (is_holiday.iloc[start_idx : end_idx + 1].sum() - is_holiday.iloc[idx])
                    if nearby_holidays == 3:
                        data.iloc[idx, data.columns.get_loc('is_sandwich')] = 3
                    elif nearby_holidays == 2:
                        data.iloc[idx, data.columns.get_loc('is_sandwich')] = 2
                    else:
                        data.iloc[idx, data.columns.get_loc('is_sandwich')] = 0
            return data

        def get_sandwich_score_for_dates(self, date, kor_holidays) -> int:
            """특정 날짜를 받아와서 앞뒤 날짜를 구하고, 샌드위치 점수 계산"""
            # 하루씩
            prev_date, next_date = date - timedelta(days = 1), date + timedelta(days = 1)
            prev_hol, next_hol = self.check_holidays(prev_date, kor_holidays), self.check_holidays(next_date, kor_holidays) # T/F Bool
            if prev_hol and next_hol: # 바로 다음 날들이 휴일이라면
                return 3
            days_offsets = [-2, -1, 1, 2] # 앞뒤로 이틀 살펴보기
            nearby_holidays = sum(self.check_holidays(date + timedelta(days = d), kor_holidays) for d in days_offsets)
            if nearby_holidays == 3: # 앞뒤 4일 중에 3일이 휴일이면
                return 2
            elif nearby_holidays == 2: # 앞뒤 4일 중에 2일이 휴일이면
                return 1
            else:
                return 0

        def get_month_weights(self, data = None, monthly_weights = monthly_weights):
            """월별 가중치 부여"""
            # 데이터프레임 들어오면
            if data is not None:
                data['month_weight'] = data['month'].map(monthly_weights)
                return data

        def get_week_weights(self, data = None, weekly_weights = weekly_weights):
            """요일별 가중치 부여"""
            # 데이터프레임 들어오면
            if data is not None:
                data['week_weight'] = data['weekday'].map(weekly_weights)
                return data

        def get_prev_days(self, data, test_df = None, date = None, menu = None, howmany = 7):
            """
            일요일 날짜 받아와서 직전 주차의 일-토 매출수량 평균 계산
            주의 - test data에서 생성할 때는 참고할 데이터와 붙여넣을 데이터가 다름
            data : 참고할 데이터
            test_df : 참고할 데이터
            """
            if test_df is None:
                # 혹시 모르니까 검증
                if date.weekday() == 6:
                    # 이전 날짜들
                    prev_start = date - timedelta(days = howmany)
                    prev_end = date - timedelta(days = 1)
                    prev_data = data[(data['영업일자'] >= prev_start) & (data['영업일자'] <= prev_end) & (data['영업장명_메뉴명'] == menu)]
                    prev_avg = prev_data['매출수량'].mean()
                    prev_sd = prev_data['매출수량'].std()
                    # 첫 주 0으로 처리
                    if pd.isna(prev_avg):
                        prev_avg = 0
                    if pd.isna(prev_sd):
                        prev_sd = 0
                    week_end = date + timedelta(days = 6)
                    curr_mask = (data['영업일자'] >= date) & (data['영업일자'] <= week_end) & (data['영업장명_메뉴명'] == menu)
                    colname_mean = f"prev_avg_{howmany}"
                    colname_sd = f"prev_sd_{howmany}"
                    data.loc[curr_mask, colname_mean] = prev_avg
                    data.loc[curr_mask, colname_sd] = prev_sd
                    return data
                else:
                    return np.nan

            # test data라면
            else:
                # 혹시 모르니까 검증
                if date.weekday() == 6:
                    # 이전 날짜들
                    prev_start = date - timedelta(days = howmany)
                    prev_end = date - timedelta(days = 1)
                    prev_data = test_df[(test_df['영업일자'] >= prev_start) & (test_df['영업일자'] <= prev_end) & (test_df['영업장명_메뉴명'] == menu)]
                    prev_avg = prev_data['매출수량'].mean()
                    prev_sd = prev_data['매출수량'].std()
                    # 첫 주 0으로 처리
                    if pd.isna(prev_avg):
                        prev_avg = 0
                    if pd.isna(prev_sd):
                        prev_sd = 0
                    week_end = date + timedelta(days = 6)
                    curr_mask = (data['영업일자'] >= date) & (data['영업일자'] <= week_end) & (data['영업장명_메뉴명'] == menu)
                    colname_mean = f"prev_avg_{howmany}"
                    colname_sd = f"prev_sd_{howmany}"
                    data.loc[curr_mask, colname_mean] = prev_avg
                    data.loc[curr_mask, colname_sd] = prev_sd
                    return data
                else:
                    return np.nan


        # train, test 공통
        def make_fund_variables(self, data, month_to_season = month_to_season):
            # 영업일자 -> datetime
            data['영업일자'] = pd.to_datetime(data['영업일자'])

            # 연, 월, 일, 요일 분리
            data['year'] = data['영업일자'].dt.year
            data['month'] = data['영업일자'].dt.month
            data['day'] = data['영업일자'].dt.day
            data['weekday'] = data['영업일자'].dt.day_name()
            data['weekday_enc'] = data['영업일자'].dt.weekday

            # 계절 변수 생성
            data['season'] = data['month'].map(month_to_season)

            # 연도 차이 변수 생성
            data['year_enc'] = data['year'] - 2023

            # 월, 일, 요일 사이클릭 변환
            data['month_sin'] = np.sin(2 * np.pi * data['month'] / 12)
            data['month_cos'] = np.cos(2 * np.pi * data['month'] / 12)

            data['day_sin'] = np.sin(2 * np.pi * data['day'] / 31)
            data['day_cos'] = np.cos(2 * np.pi * data['day'] / 31)

            data['weekday_sin'] = np.sin(2 * np.pi * data['weekday_enc'] / 7)
            data['weekday_cos'] = np.cos(2 * np.pi * data['weekday_enc'] / 7)

            # 공휴일 확인
            kor_holidays = self.update_kor_holidays()
            check_holiday = data['영업일자'].dt.date.isin(kor_holidays)
            check_weekend = data['weekday'].isin(['Saturday', 'Sunday'])
            data['is_holiday'] = (check_holiday | check_weekend).astype(int) # 공휴일 + 주말
            data['holiday_name'] = data['영업일자'].dt.date.map(kor_holidays)

            return data

        # train의 입력 데이터
        def make_variables_train(self, data):
            data = self.make_fund_variables(data)
            kor_holidays = self.update_kor_holidays()

            ### 샌드위치 데이
            data = self.get_sandwich_score(data, 'is_holiday')

            # 샌드위치 - 첫날
            first = data['영업일자'].min()
            data.loc[data['영업일자'] == first, 'is_sandwich'] = self.get_sandwich_score_for_dates(first, kor_holidays)
            second = data['영업일자'].min() + timedelta(days = 1)
            data.loc[data['영업일자'] == second, 'is_sandwich'] = self.get_sandwich_score_for_dates(second, kor_holidays)

            # 샌드위치 - 마지막 날
            last = data['영업일자'].max()
            data.loc[data['영업일자'] == last, 'is_sandwich'] = self.get_sandwich_score_for_dates(last, kor_holidays)
            before = data['영업일자'].max() - timedelta(days = 1)
            data.loc[data['영업일자'] == before, 'is_sandwich'] = self.get_sandwich_score_for_dates(before, kor_holidays)

            # 샌드위치 포함한 공휴일
            data['is_holiday_sandwich'] = data['is_holiday'].astype(int) | (data['is_sandwich'] > 0).astype(int)

            ### 월별 가중치
            data = self.get_month_weights(data, monthly_weights)

            ### 요일별 가중치
            data = self.get_week_weights(data, weekly_weights)

            ### 직전 주차 평균
            sundays = data[data['weekday'] == "Sunday"][["영업일자", "영업장명_메뉴명"]].copy()
            for _, row in sundays.iterrows():
                date = row['영업일자']
                menu = row['영업장명_메뉴명']
                data = self.get_prev_days(data = data, date = date, menu = menu, howmany = 7)
                data = self.get_prev_days(data = data, date = date, menu = menu, howmany = 14)
                data = self.get_prev_days(data = data, date = date, menu = menu, howmany = 21)

            ### 영업장명, 메뉴명 분리
            if '영업장명_메뉴명' in data.columns:
                data[['영업장명', '메뉴명']] = data['영업장명_메뉴명'].str.split('_', expand = True)

            ### 음수 처리
            negative = data[data['매출수량'] < 0]

            for idx, row in negative.iterrows():
                num = row['매출수량']
                if num < -10:
                    date = row['영업일자']
                    menu = row['영업장명_메뉴명']
                    prev_date = pd.to_datetime(date) - pd.Timedelta(days = 1)
                    prev_row = data[(data['영업일자'] == prev_date) & (data['영업장명_메뉴명'] == menu)]

                    if prev_row.iloc[0]["매출수량"] >= abs(num):
                        data.loc[prev_row.index[0], '매출수량'] += num

            # 남은 건 전부 0으로
            data.loc[data['매출수량'] < 0, '매출수량'] = 0

            return data

        # 예측하고자 하는 날들
        def make_variables_test(self, date, test_df, predict):
            """
            date : 최종 날짜 (입력 7일 중 가장 마지막) - TimeStamp
            test_df : 예측할 때 참고해올 데이터 -> 이거로 직전 주차 평균 생성
            """
            date = pd.to_datetime(date)
            future_dates = [date + timedelta(days = i + 1) for i in range(predict)]
            future_df = pd.DataFrame({'영업일자' : future_dates})

            menus_df = (test_df[['영업장명_메뉴명']].drop_duplicates().reset_index(drop = True))
            future_df = future_df.merge(menus_df, how='cross')

            kor_holidays = self.update_kor_holidays()

            # 기본적인 변수들
            future_df = self.make_fund_variables(future_df)

            future_df['영업일자'] = pd.to_datetime(future_df['영업일자']).dt.normalize()

            # 샌드위치
            future_df['is_sandwich'] = future_df['영업일자'].apply(lambda d: self.get_sandwich_score_for_dates(d, kor_holidays))

             # 샌드위치 포함한 공휴일
            future_df['is_holiday_sandwich'] = future_df['is_holiday'].astype(int) | (future_df['is_sandwich'] > 0).astype(int)

            # 월별 가중치
            future_df = self.get_month_weights(future_df, monthly_weights)

            # 요일별 가중치
            future_df = self.get_week_weights(future_df, weekly_weights)

            # 직전 주차 평균 -> 이거는 test 까지 받아오고 생각해야 함..
            sundays =  future_df.loc[future_df['weekday'] == "Sunday", ['영업일자', '영업장명_메뉴명']].copy()
            for _, row in sundays.iterrows():
                date = row['영업일자']
                menu = row['영업장명_메뉴명']
                future_df = self.get_prev_days(data = future_df, test_df = test_df, date = date, menu = menu, howmany = 7)
                future_df = self.get_prev_days(data = future_df, test_df = test_df, date = date, menu = menu, howmany = 14)
                future_df = self.get_prev_days(data = future_df, test_df = test_df, date = date, menu = menu, howmany = 21)

            return future_df

In [ ]:
# 그냥 전부 만들면 돼
mv = Make_Variables()
data = mv.make_variables_train(data)

In [ ]:
print("기존 공휴일 : ", data['is_holiday'].sum().item())
print("샌드위치 : ", data['is_sandwich'].sum().item())
print("샌드위치 포함 공휴일 : ", data['is_holiday_sandwich'].sum().item())

기존 공휴일 :  33389
샌드위치 :  579
샌드위치 포함 공휴일 :  33968


In [ ]:
import pickle
data.to_pickle("/content/drive/MyDrive/3. Grad School/LG Aimers/DATA/train_data_new.pickle")

#### 저장된 데이터 불러오기

In [ ]:
cols =  ["year_enc", "month_sin", "month_cos", "day_sin", "day_cos", "weekday_sin", "weekday_cos", "season", "is_holiday", "is_sandwich", "is_holiday_sandwich", "month_weight", "week_weight",
         "prev_avg_7", "prev_avg_14", "prev_avg_21", "prev_sd_7", "prev_sd_14", "prev_sd_21"]
enc_cols = ["season", "is_holiday", "is_holiday_sandwich"]
# lstm에서는 minmaxscaling 해주는 게 안전
num_cols = ["is_sandwich", "month_weight", "week_weight", "prev_avg_7", "prev_avg_14", "prev_avg_21", "prev_sd_7", "prev_sd_14", "prev_sd_21"]

#### 매출 여부 (분류)

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import TimeSeriesSplit
! pip install category_encoders
from category_encoders import TargetEncoder
from sklearn.metrics import f1_score
from typing import Dict, List, Optional, Any
from collections import defaultdict
import pickle
import joblib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 1.1 MB/s eta 0:00:00


In [ ]:
class ClassificationModel():
    def __init__(self, data = None, cols = None, enc_cols = None, model_path = None):
        self.data = data
        self.cols = cols
        self.enc_cols = enc_cols

    def fit_model_cv(self, data : pd.DataFrame, cols : List[str], enc_cols : List[str]) -> pd.DataFrame:
        """
        data - 전체 dataset
        cols - 전체 변수들
        enc_cols - 인코딩 진행할 변수들 (범주형)
        """
        oof_parts = []

        for menu, group_df in data.groupby("영업장명_메뉴명"):

            group_df = group_df.sort_values('영업일자')

            # 나중에 oof 저장할 때 사용할 것들
            org_idx = group_df.index
            oof_proba = np.full(len(group_df), np.nan)

            # x, y 분리
            x = group_df[cols]
            y = group_df["매출_여부"]

            # time series split
            tscv = TimeSeriesSplit(n_splits = 3)

            # 각 split 별로
            for fold, (train_idx, val_idx) in enumerate(tscv.split(x)):

                x_train, y_train = x.iloc[train_idx], y.iloc[train_idx]
                x_val, y_val = x.iloc[val_idx], y.iloc[val_idx]

                # 수량 전부 0이거나 0 아닌 날 없으면 학습 불가
                if y_train.nunique() < 2:
                    print(f"{menu} 학습 불가")
                    continue

                # 범주형 변수 인코딩
                target_encoder = TargetEncoder()
                target_encoder.fit(x_train[enc_cols], y_train)

                x_train = pd.concat([
                    target_encoder.transform(x_train[enc_cols]),
                    x_train[[c for c in cols if c not in enc_cols]]
                ], axis = 1)

                x_val = pd.concat([
                    target_encoder.transform(x_val[enc_cols]),
                    x_val[[c for c in cols if c not in enc_cols]]
                ], axis = 1)

                # 모델 설정
                xgb_model = XGBClassifier(random_state = 1471)

                # 모델 학습
                xgb_model.fit(x_train, y_train)
                oof_proba[val_idx] = xgb_model.predict_proba(x_val)[:, 1]

            part = pd.DataFrame({
                "index": org_idx.values,
                "영업장명_메뉴명": menu,
                "y_true": group_df["매출_여부"].values,
                "y_proba": oof_proba
            }).set_index("index")

            oof_parts.append(part)

        oof_df = pd.concat(oof_parts).sort_index()
        return oof_df


    def tuning_cv(self, data, cols, enc_cols, param_grid):
        """
        메뉴별로 하이퍼파라미터 튜닝 진행 (성능 보고.. 추가 진행? )
        """
        best_params_by_menu = {}

        return None


    def tune_threshold(self, oof_df : pd.DataFrame, metric = 'f1') -> Dict[str, float]:
        """
        메뉴별로 OOF 데이터프레임을 받아와서
        threshold를 최적화
        metric으로는 f1 사용
        """
        best_thresholds = {}

        for menu, group_df in oof_df.groupby("영업장명_메뉴명"):
            y_true = group_df["y_true"].values
            y_proba = group_df["y_proba"].values

            # 초기값 진행
            best_score, best_thr = -1, 0.5
            for thr in np.linspace(0.05, 0.95, 51):
                y_pred = (y_proba >= thr).astype(int)
                score = f1_score(y_true, y_pred, zero_division = 0)
                if score > best_score:
                    best_score = score
                    best_thr = thr

            best_thresholds[menu] = best_thr

        return best_thresholds


    def get_final_model(self, data, cols, enc_cols, thresholds, hyperparameters_dict = None) -> Dict[str, Dict[str, Any]]:
        """
        메뉴별로 튜닝된 하이퍼파라미터, threshold를 반영하여
        최종 모델 적합
        """
        models = {}

        for menu, group_df in data.groupby("영업장명_메뉴명"):

            group_df = group_df.sort_values('영업일자')

            # 범주형 변수 처리 - 전체 데이터로
            target_encoder_full = TargetEncoder()
            group_df[enc_cols] = target_encoder_full.fit_transform(group_df[enc_cols], group_df['매출_여부'])

            # x, y 분리
            x_full = group_df[cols]
            y_full = group_df["매출_여부"]

            # 하이퍼파라미터
            hyperparameters = hyperparameters_dict.get(menu, {}) if hyperparameters_dict else {}

            # 모델 설정
            xgb_model_full = XGBClassifier(random_state = 1471, **hyperparameters)

            # 모델 학습
            xgb_model_full.fit(x_full, y_full)

            models[menu] = {
                "model" : xgb_model_full,
                "encoder" : target_encoder_full,
                "threshold" : thresholds.get(menu, 0.5)}

        return models

    def fit_whole_model(self, data, cols, enc_cols) -> Dict[str, Dict[str, Any]]:
        oof_df = self.fit_model_cv(data, cols, enc_cols)
        print("CV 완료!")
        thresholds = self.tune_threshold(oof_df)
        print("threshold 탐색 완료!")
        models = self.get_final_model(data, cols, enc_cols, thresholds)
        return models

    def save_cls_model(self, models, model_path):
        joblib.dump(models, model_path)
        print("모델 저장 완료!")

    def load_saved_model(self, model_path):
        models = joblib.load(model_path)
        return models

In [ ]:
classification = ClassificationModel()
model_path = '/content/drive/MyDrive/3. Grad School/LG Aimers/Models/Trial 3/cls_models.pkl'

data_zero = data.copy()
data_zero['매출_여부'] = data_zero['매출수량'].apply(lambda x:1 if x > 0 else 0)

models = classification.fit_whole_model(data = data_zero, cols = cols, enc_cols = enc_cols)
classification.save_cls_model(models, model_path)

In [ ]:
# 다시 불러오기
model_path = '/content/drive/MyDrive/3. Grad School/LG Aimers/Models/Trial 6/cls_models.pkl'
classification = ClassificationModel()
models_class = classification.load_saved_model(model_path)

In [ ]:
    def predict_reg_model(self, test_df, trained_model, test_prefix : str, cols : list, enc_cols : list, data, lookback = 28, predict = 7):
        """
        Input : test_df - test data, trained_model - {~~}, original_data : train할 때 사용한 데이터 (data)
        Output : [영업일자, 영업장명_메뉴명, 매출수량] DataFrame
        """
        results = []

        # 모델 불러오기
        model = trained_model["model"]
        encoder = trained_model["encoder"]

        # 변수 추가하기
        mv = Make_Variables()

        for store_menu, store_df in test_df.groupby(['영업장명_메뉴명']):

            store_df['영업일자'] = pd.to_datetime(store_df['영업일자'])
            store_df_sorted = store_df.sort_values('영업일자')
            last_date = store_df_sorted['영업일자'].iloc[-1]

            future_df = mv.make_variables_test(date = last_date, test_df = store_df, original_data = data, predict = 7)
            if enc_cols:
                future_df[enc_cols] = encoder.transform(future_df[enc_cols])

            # 사용할 변수만
            future_df = future_df[cols]

            # 로그 변환 처리
            predicted = model.predict(future_df)

            # 예측일자: TEST_00+1일 ~ TEST_00+7일
            pred_dates = [f"{test_prefix}+{i+1}일" for i in range(predict)]

            for d, val in zip(pred_dates, predicted):
                results.append({
                    '영업일자': d,
                    '영업장명_메뉴명': store_menu,
                    '매출수량': val
                })

        return pd.DataFrame(results)

#### 매출 예측 (회귀)

In [ ]:
from xgboost import XGBRegressor
# ! pip install category_encoders
from category_encoders import TargetEncoder
from sklearn.model_selection import TimeSeriesSplit
from collections import defaultdict
from itertools import product
import pickle
import joblib

In [ ]:
class RegressionModel():
    def __init__(self, data = None, cols = None, enc_cols = None, model_path = None):
        self.data = data
        self.cols = cols
        self.enc_cols = enc_cols

    def smape_score(self, y_true, y_pred, eps = 1e-8):
        y_true = np.asarray(y_true, dtype=float)
        y_pred = np.asarray(y_pred, dtype=float)
        denom = np.abs(y_true) + np.abs(y_pred)
        denom = np.where(denom < eps, eps, denom)
        return 200.0 * np.mean(np.abs(y_true - y_pred) / denom)

    def tuning_cv(self, data : pd.DataFrame, cols : List[str], enc_cols : List[str], param_grid = None) -> Dict[str, Dict]:
        """
        메뉴별로 하이퍼파라미터 튜닝 진행
        """
        best_params_by_menu = {}

        if param_grid is None:
            param_grid = {
                "min_child_weight" : [1, 5],
                "max_depth" : [5, 6, 7],
                "subsample" : [0.7, 0.9],
                "n_estimators" : [100, 150],
                "learning_rate" : [0.03, 0.05]}

        keys = list(param_grid.keys())
        combos = list(product(*[param_grid[k] for k in keys]))

        for menu, group_df in data.groupby("영업장명_메뉴명"):

            group_df = group_df.sort_values("영업일자")

            x = group_df[cols]
            y = group_df["매출수량"]

            # time series split
            tscv = TimeSeriesSplit(n_splits = 3)

            # 초기값
            best_smape = np.inf
            best_params = None

            for values in combos:
                params = dict(zip(keys, values))

                base_params = {"random_state" : 1478}
                base_params.update(params)

                fold_smapes : List[float] = []

                for fold, (train_idx, val_idx) in enumerate(tscv.split(x)):

                    x_train, y_train = x.iloc[train_idx], y.iloc[train_idx]
                    x_val, y_val = x.iloc[val_idx], y.iloc[val_idx]

                    # 데이터 수 적으면 학습 불가
                    if len(x_train) < 20:
                        print(f"{menu} 튜닝 불가)")
                        continue

                    # 범주형 변수 인코딩
                    target_encoder = TargetEncoder()
                    target_encoder.fit(x_train[enc_cols], y_train)

                    x_train = pd.concat([
                        target_encoder.transform(x_train[enc_cols]),
                        x_train[[c for c in cols if c not in enc_cols]]
                    ], axis = 1)

                    x_val = pd.concat([
                        target_encoder.transform(x_val[enc_cols]),
                        x_val[[c for c in cols if c not in enc_cols]]
                    ], axis = 1)

                    # 모델 설정
                    xgb_model = XGBRegressor(**base_params)

                    # 모델 학습
                    xgb_model.fit(x_train, y_train)

                    pred = xgb_model.predict(x_val)
                    smape = self.smape_score(y_val, pred)
                    fold_smapes.append(smape)

                mean_smape = float(np.mean(fold_smapes))

                if mean_smape < best_smape:
                    best_smape = mean_smape
                    best_params = base_params

            if best_params is not None:
                best_params_by_menu[menu] = best_params

        return best_params_by_menu


    def get_final_model(self, data, cols, enc_cols, hyperparameters_dict : dict[str, dict[str, Any]] = None)  -> Dict[str, Dict[str, Any]]:
        """
        data, validation_reg - train, validation dataset
        cols - 전체 변수들
        enc_cols - 인코딩 진행할 변수들 (범주형)
        """
        models = {}

        for menu, group_df in data.groupby("영업장명_메뉴명"):

            group_df = group_df.sort_values("영업일자")

            # 데이터 수 적으면 학습 불가
            if len(group_df) < 10:
                print(f"{menu} 학습 불가")
                continue

            # 범주형 변수 처리
            target_encoder_full = TargetEncoder()
            group_df[enc_cols] = target_encoder_full.fit_transform(group_df[enc_cols], group_df['매출수량'])

            # x, y 분리
            x_full = group_df[cols]
            y_full = group_df["매출수량"]

            # 하이퍼파라미터 불러오기
            params = hyperparameters_dict.get(menu, {}) if hyperparameters_dict else {}

            base_params = {"random_state" : 1478}
            base_params.update(params)

            # 모델 설정
            xgb_model_full = XGBRegressor(**base_params)

            # 모델 학습
            xgb_model_full.fit(x_full, y_full)

            models[menu] = {
                "model" : xgb_model_full,
                "hyperparameters" : base_params,
                "encoder" : target_encoder_full
            }

        return models

    def fit_whole_model(self, data, cols, enc_cols) -> Dict[str, Dict[str, Any]]:
        best_params_by_menu = self.tuning_cv(data, cols, enc_cols)
        print("튜닝 완료!")
        models = self.get_final_model(data, cols, enc_cols, best_params_by_menu)
        return models

    def save_reg_model(self, models, model_path):
        joblib.dump(models, model_path)
        print("모델 저장 완료!")

    def load_saved_model(self, model_path):
        models = joblib.load(model_path)
        return models

In [ ]:
regression = RegressionModel()
model_path = '/content/drive/MyDrive/3. Grad School/LG Aimers/Models/Trial 6/reg_models.pkl'

data_notzero = data[data['매출수량'] > 0]

models = regression.fit_whole_model(data = data_notzero, cols = cols, enc_cols = enc_cols)
regression.save_reg_model(models, model_path)

튜닝 완료!
모델 저장 완료!


In [ ]:
# 다시 불러오기
model_path = '/content/drive/MyDrive/3. Grad School/LG Aimers/Models/Trial 6/reg_models.pkl'
regression = RegressionModel()
models_reg = regression.load_saved_model(model_path)

#### 매출 예측 (시계열)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import TimeSeriesSplit
import random
import glob
import joblib

import torch
import torch.nn as nn
from tqdm import tqdm

In [ ]:
### Random Seed & Parameters
def set_seed(seed = 1471):
    random.seed(seed) # 일반 seed
    np.random.seed(seed) # numpy 난수 고정
    torch.manual_seed(seed) # CPU 난수 고정
    os.environ["PYTHONHASHSEED"] = str(seed)

    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

set_seed(1478)

##### Loss Functions

In [ ]:
# weight 직접 입력해서 반영 가능
class WeightedSMAPELoss(nn.Module):
    def __init__(self, eps=1e-8):
        super().__init__()
        self.eps = eps

    def forward(self, y_pred, y_true, w=None):
        num = (y_pred - y_true).abs()
        den = (y_pred.abs() + y_true.abs()).clamp(min=self.eps)
        smape = 2.0 * num / (den)

        mask = (y_true != 0).float()
        if w is None:
            w = torch.ones_like(y_true)
        else:
            if w.dim() == 1:
                w = w.view(-1, 1)
            w = w.expand_as(y_true)

        w_mask = w * mask
        denom = w_mask.sum().clamp(min=self.eps)
        loss = (smape * w_mask).sum() / denom
        return loss

# weighted huberLoss
class WeightedHuberLoss(nn.Module):
    def __init__(self, delta = 1.0, zeros : bool = False, eps = 1e-8):
        """
        delta -L2 -> L1로 전환되는 임계값
        zeros - y_true = 0인 샘플 제외할지 여부
        eps - 분모가 0이 되는 것을 방지하기 위한 작은 값
        """
        super().__init__()
        self.delta = float(delta)
        self.zeros = bool(zeros)
        self.eps = float(eps)

    def forward(self, y_pred, y_true, w = None):
        """
        w -  (N, ) 형태로 된 가중치
        """
        error = y_pred - y_true
        abs_error = error.abs()
        huber = torch.where(
            abs_error <= self.delta,
            0.5 * error**2,
            self.delta * (abs_error - 0.5 * self.delta),
        )

        if self.zeros:
            mask = (y_true != 0).float()
        else:
            mask = torch.ones_like(y_true)

        if w is None:
            w_full = torch.ones_like(y_true)
        else:
            if w.dim() == 1:
                w = w.view(-1, 1) # (N, 1)로 확장
            w_full = w.expand_as(y_true).float()

        w_mask = w_full * mask
        denom = w_mask.sum().clamp(min=self.eps)
        return (huber * w_mask).sum() / denom

# SMAPE + Huber Loss
class CombinationLoss(nn.Module):
    def __init__(self, losses, weights):
        """
        losses - 결합할 손실 리스트
        weights - 각 손실의 가중치 리스트
        """
        super().__init__()
        if not losses or len(losses) != len(weights):
            raise ValueError("loss 배열과 weight 배열의 길이가 다릅니다.")
        self.losses = nn.ModuleList(losses)

        weight_tensor = torch.tensor(weights, dtype=torch.float32)
        self.weights = weight_tensor / weight_tensor.sum()

    def forward(self, y_pred, y_true, w = None):
        """
        w - 공통 가중치
        """
        total_loss = 0.0
        device_weights = self.weights.to(y_pred.device)
        for i, lf in enumerate(self.losses):
            loss = lf(y_pred, y_true, w)
            total_loss += device_weights[i] * loss
        return total_loss

In [ ]:
def build_menu_weights(
    df: pd.DataFrame,
    menu_key: str = "영업장명_메뉴명",
    target_col: str = "매출수량",
    dispersion: str = "p95-p5",   # "range" | "iqr" | "std" | "p95-p5"
    alpha: float = 1.0,            # 민감도 (↑ 크면 가중치 차이 커짐)
    clip: tuple = (0.25, 4.0),     # 과도한 비율 방지
    eps: float = 1e-6
) -> dict:
    """
    메뉴별 난이도(분산/범위) 기반 가중치 (간극 ↑ -> 가중치 ↑)
    반환: {menu_key -> weight}
    """
    disp_vals = {}
    for k, g in df.groupby(menu_key):
        y = g[target_col].to_numpy()
        if len(y) == 0:
            disp = 0.0
        else:
            if dispersion == "range":
                disp = float(np.max(y) - np.min(y))
            elif dispersion == "iqr":
                q75, q25 = np.percentile(y, [75, 25])
                disp = float(q75 - q25)
            elif dispersion == "std":
                disp = float(np.std(y))
            else:  # "p95-p5"
                q95, q5 = np.percentile(y, [95, 5])
                disp = float(q95 - q5)
        disp_vals[k] = disp

    # 분포 → 가중치 (단조증가), 평균 1로 정규화
    disp_series = pd.Series(disp_vals)
    # 0도 있을 수 있으니 +eps, 로그스케일 쓰고 싶으면 바꿔도 됨
    raw_w = (disp_series + eps) ** alpha
    w = raw_w / raw_w.mean()

    # 과도한 값 클램프 (옵션)
    w = w.clip(lower=clip[0], upper=clip[1])
    return w.to_dict()

def build_store_weights(
    df: pd.DataFrame,
    store_col: str = "영업장명",
    menu_col: str = "메뉴명",      # 없으면 아래에서 자동 fallback
    combined_key: str = "영업장명_메뉴명",
    beta: float = 1.0,             # 메뉴 가짓수 민감도
    clip: tuple = (0.5, 3.0)
) -> dict:
    """
    음식점별 메뉴 가짓수 기반 가중치 (가짓수 ↑ -> 가중치 ↑)
    반환: {store -> weight}
    """
    if (store_col in df.columns) and (menu_col in df.columns):
        n_menu = df.groupby(store_col)[menu_col].nunique()
    else:
        # fallback: combined_key에서 store를 추출 (최초 '_' 기준 split)
        def get_store(x: str) -> str:
            # 필요하면 사용자 규칙에 맞게 수정
            return str(x).split("_", 1)[0]
        tmp = df[[combined_key]].copy()
        tmp[store_col] = tmp[combined_key].astype(str).map(get_store)
        # store별 unique combined_key 개수 = 메뉴 가짓수로 근사
        n_menu = tmp.groupby(store_col)[combined_key].nunique()

    raw_w = (n_menu.astype(float) + 1e-6) ** beta
    w = raw_w / raw_w.mean()
    w = w.clip(lower=clip[0], upper=clip[1])
    return w.to_dict()

##### LSTMModel

In [ ]:
class MultiOutputLSTM(nn.Module):
        def __init__(self, input_dim = 1, hidden_dim = 256, num_layers = 4, output_dim = 7, dropout = 0.4):
            """ 7개 값 예측 (PREDICT 만큼의 날짜의 값을 예측하고자 함)"""
            super(MultiOutputLSTM, self).__init__()
            self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first = True, dropout = dropout)
            self.fc = nn.Linear(hidden_dim, output_dim)

        def forward(self, x):
            out, _ = self.lstm(x)
            return self.fc(out[:, -1, :]) # 마지막 시점 출력만 선택해서 fc에 넣음 -> (batch * output_dim)

In [ ]:
class LSTMModel():
    def __init__(self, data = None, cols = None, enc_cols = None, num_cols = None, scaler = None,
                 lookback = 28, predict = 7, hidden_dim = 256, num_layers = 4, device = "cuda", epochs = 200, batch_size = 16, dropout = 0.4,
                 delta = 0.1, val_ratio = 0.2, horizon_weight_mode = "linear",
                 lr = 0.04, losses = None, loss_weights = [0.5, 0.5], menu_weights = None):
        self.data = data
        self.cols = cols
        self.enc_cols = enc_cols
        self.scaler = scaler

        self.lookback = lookback
        self.predict = predict
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.device = torch.device(device)
        self.epochs = epochs
        self.batch_size = batch_size
        self.dropout = dropout

        self.delta = delta
        self.val_ratio = val_ratio
        self.horizon_weight_mode = horizon_weight_mode
        self.lr = float(lr)
        self.losses = losses
        self.loss_weights = loss_weights
        self.menu_weights = menu_weights

    # enc_cols는 LabelEncoding
    def label_encoding_lstm(self, data, enc_cols):
        encoders = {}
        for col in enc_cols:
            if data[col].dtype == 'object' or data[col].dtype.name == 'bool' or data[col].dtype.name == 'category':
                le = LabelEncoder()
                data[col] = le.fit_transform(data[col])
                encoders[col] = le
        return data, encoders

    def minmax_scaling_features(self, data, num_cols):
        scaler = MinMaxScaler()
        data[num_cols] = scaler.fit_transform(data[num_cols])
        return data, scaler

    # 매출수량은 MinMaxScaling
    def minmax_scaling_target(self, data):
        scaler = MinMaxScaler()
        data['매출수량'] = scaler.fit_transform(data[['매출수량']])
        return data, scaler

    def build_horizon_weights(self, predict = 7):
        """
        수평선(예측일) 가중치 벡터 생성.
        - 'linear': 1..predict 선형 가중 후 정규화
        - 'uniform': 동일 가중
        """
        if self.horizon_weight_mode == "uniform":
            w = torch.ones(predict)
        else:
            # 기본: 뒤쪽일수록 더 큰 가중
            w = torch.arange(1, predict + 1).float()
        w = w / w.sum()
        return w

    def compute_smape(self, y_pred, y_true, eps=1e-8):
        """정규화 스케일에서의 SMAPE( y_true==0 은 마스킹 )"""
        num = (y_pred - y_true).abs()
        den = (y_pred.abs() + y_true.abs()).clamp(min=eps)
        smape = 2.0 * num / den
        mask = (y_true != 0).float()
        denom = mask.sum().clamp(min=eps)
        return (smape * mask).sum().item() / denom.item()

    def _tss_last_split_indices(self, n_samples: int, n_splits: int = 3):
        """
        마지막 TimeSeriesSplit fold의 (train_idx, val_idx)만 반환.
        누수 방지를 위해 gap = lookback + predict - 1 권장.
        """
        idx = np.arange(n_samples)
        tscv = TimeSeriesSplit(n_splits=n_splits, gap=self.lookback + self.predict - 1)
        tr_idx, va_idx = None, None
        for tr, va in tscv.split(idx):
            tr_idx, va_idx = tr, va
        return tr_idx, va_idx

    def _make_windows(self, x: np.ndarray, y: np.ndarray):
        """
        구간 내부에서만 윈도우 생성 (누수 없음)
        x: (T, F), y: (T,)
        return: x:(N, lookback, F), y:(N, predict) or (None, None)
        """
        T = len(x)
        xs, ys = [], []
        for i in range(T - self.lookback - self.predict + 1):
            xs.append(x[i:i+self.lookback])
            ys.append(y[i+self.lookback:i+self.lookback+self.predict])
        if not xs:
            return None, None
        return np.stack(xs).astype(np.float32), np.stack(ys).astype(np.float32)

#############################################################################
    def train_lstm(self, train_df, cols, enc_cols, num_cols,
                   device, epochs, batch_size, lr, dropout,
                   losses, loss_weights,
                   n_splits : int = 3, print_every = 50):
        """
        영업장, 메뉴별로 LSTM 모델 훈련, 각각을 trained_models에 저장
        Loss - CombinationLoss([WeightedSMAPELoss(), WeightedHuberLoss()], [0.5, 0.5]
        """
        trained_models = {}
        horizon_w = self.build_horizon_weights(self.predict).to(device)

        # Loss 구성
        comb_loss = CombinationLoss(losses = losses, weights = loss_weights).to(device)

        # store_menu : 영업장명_메뉴명 / group : 나머지 데이터
        for store_menu, group in tqdm(train_df.groupby(["영업장명_메뉴명"]), desc = "Training LSTM"):

            # 날짜 순으로 정렬해서 데이터가 너무 적으면 -> 학습 생략
            store_train = group.sort_values("영업일자").copy()
            if len(store_train) < self.lookback + self.predict:
                continue

            # ===== TimeSeriesSplit: 마지막 폴드를 검증으로 사용 =====
            tr_idx, va_idx = self._tss_last_split_indices(len(store_train), n_splits=n_splits)
            df_tr = store_train.iloc[tr_idx].copy()
            df_va = store_train.iloc[va_idx].copy()

            # ===== 인코더/스케일러는 train(df_tr)에만 fit, df_tr/df_va에 transform (누수 방지) =====
            encoders = {}
            if enc_cols:
                for c in enc_cols:
                    le = LabelEncoder()
                    df_tr[c] = le.fit_transform(df_tr[c].astype(str))
                    # val에서 미지 카테고리 안전 처리
                    cls2idx = {cls: i for i, cls in enumerate(le.classes_)}
                    df_va[c] = df_va[c].astype(str).map(lambda v: cls2idx.get(v, -1))
                    encoders[c] = le

            features_scaler = None
            if num_cols:
                features_scaler = MinMaxScaler()
                df_tr[num_cols] = features_scaler.fit_transform(df_tr[num_cols])
                df_va[num_cols] = features_scaler.transform(df_va[num_cols])

            target_scaler = MinMaxScaler()
            df_tr[['매출수량']] = target_scaler.fit_transform(df_tr[['매출수량']])
            df_va[['매출수량']] = target_scaler.transform(df_va[['매출수량']])

            # ===== 원시 배열 =====
            X_tr = df_tr[cols].to_numpy(dtype=np.float32)
            y_tr = df_tr['매출수량'].to_numpy(dtype=np.float32)
            X_va = df_va[cols].to_numpy(dtype=np.float32)
            y_va = df_va['매출수량'].to_numpy(dtype=np.float32)

            # ===== 세그먼트 내부에서만 윈도우 생성 (경계 누수 없음) =====
            x_tr_np, y_tr_np = self._make_windows(X_tr, y_tr)
            x_va_np, y_va_np = self._make_windows(X_va, y_va)
            if x_tr_np is None or x_va_np is None:
                continue

            # 텐서 이동
            x_train = torch.tensor(x_tr_np).to(device)         # (N_tr, lookback, F)
            y_train = torch.tensor(y_tr_np).to(device)         # (N_tr, predict)
            x_val   = torch.tensor(x_va_np).to(device)
            y_val   = torch.tensor(y_va_np).to(device)

            # 모델 초기화 (영업장_메뉴별로 다른 모델)
            model = MultiOutputLSTM(input_dim = len(cols),
                                    hidden_dim = self.hidden_dim,
                                    num_layers = self.num_layers,
                                    output_dim = self.predict,
                                    dropout = dropout).to(device)

            optimizer = torch.optim.Adam(model.parameters(), lr = lr)
            best_val = float("inf")
            best_sd = None
            best_epoch = -1

            # 학습 모드로 설정
            model.train()

            # epochs 만큼 훈련
            for epoch in range(1, epochs + 1):
                model.train(True)
                perm = torch.randperm(len(x_train), device = device)
                epoch_train_loss = 0.0
                total_train_count = 0

                # idx : 랜덤하게 섞인 index들
                for i in range(0, len(x_train), batch_size):
                    batch_idx = perm[i : i+batch_size] # 배치 개수만큼 끊어서
                    x_batch, y_batch = x_train[batch_idx], y_train[batch_idx] # 배치 데이터 할당

                    weight_mat = horizon_w.unsqueeze(0).repeat(y_batch.size(0), 1)

                    output = model(x_batch) # 모델 태워서
                    loss = comb_loss(output, y_batch, w = weight_mat) # 평가하고

                    optimizer.zero_grad() # 역전파를 위한 초기화
                    loss.backward() # 역전파
                    optimizer.step() # 최적화

                    epoch_train_loss += loss.item() * y_batch.size(0)
                    total_train_count += y_batch.size(0)

                epoch_train_loss /= max(1, total_train_count)

                ### Validation
                with torch.no_grad():
                    model.eval()
                    pred_val = model(x_val)
                    weight_val = horizon_w.unsqueeze(0).repeat(y_val.size(0), 1)
                    val_loss = comb_loss(pred_val, y_val, w = weight_val).item()
                    val_smape = self.compute_smape(pred_val, y_val)

                    if epoch % 50 == 0:
                        print(f"\n[{store_menu}] Epoch {epoch} ==============================\n "
                            f"Train Loss {epoch_train_loss:.5f} | Val Loss {val_loss:.5f} | SMAPE {val_smape:.5f}")

                    if val_loss < best_val - 1e-6:
                        best_val = val_loss
                        best_sd = {k : v.detach().cpu().clone() for k, v in model.state_dict().items()}
                        best_epoch = epoch

                model.train(True)


            if best_sd is not None:
                model.load_state_dict(best_sd)

            # 모델 저장
            trained_models[store_menu] = {
                'model': model.state_dict(),
                'encoders' : encoders,
                'features_scaler' : features_scaler,
                'target_scaler': target_scaler,
                'meta' : {
                    'input_dim': len(cols),
                    'hidden_dim': self.hidden_dim,
                    'num_layers': self.num_layers,
                    'output_dim': self.predict,
                    'dropout' : self.dropout
                }
                }

        return trained_models

    def save_lstm_model_gpu(self, models, model_path):
        joblib.dump(models, model_path)
        print("GPU 버전 모델 저장 완료!")

    def save_lstm_model_cpu(self, models, model_path):
        cpu_models = {}
        for k, bundle in models.items():
            cpu_models[k] = {
            'model': {kk: vv.cpu() for kk, vv in bundle['model'].items()},  # 모델만 CPU로
            'encoders': bundle['encoders'],
            'features_scaler': bundle['features_scaler'],
            'target_scaler': bundle['target_scaler'],
            'meta' : bundle['meta']
        }
        joblib.dump(cpu_models, model_path)
        print("CPU 버전 모델 저장 완료!")

    def load_saved_model(self, model_path):
        models = joblib.load(model_path)
        return models

##### Train

In [ ]:
# 데이터 준비
features = cols + ["매출수량", "영업일자", "영업장명_메뉴명"]
dataset_lstm = data[features]

losses = [WeightedSMAPELoss(eps = 1e-8), WeightedHuberLoss(delta = 1.0, zeros = False, eps = 1e-8)]
loss_weights = [0.5, 0.5]
lstm = LSTMModel(lookback = 28, hidden_dim = 256, num_layers = 4, predict = 7, dropout = 0.3)

lookback, predict, batch_size, epochs = 28, 7, 16, 200
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
trained_lstm = lstm.train_lstm(train_df = dataset_lstm, cols = cols, enc_cols = enc_cols, num_cols = num_cols,
                               device = device, epochs = epochs, batch_size = batch_size, lr = 1e-3, dropout = 0.4,
                               losses = losses, loss_weights = loss_weights, n_splits = 3, print_every = 50)

Training LSTM:   0%|          | 0/193 [00:00<?, ?it/s]


[('느티나무 셀프BBQ_1인 수저세트',)] Epoch 50 ==============================
 Train Loss 0.19378 | Val Loss 0.35608 | SMAPE 0.69510

[('느티나무 셀프BBQ_1인 수저세트',)] Epoch 100 ==============================
 Train Loss 0.11939 | Val Loss 0.38836 | SMAPE 0.76970

[('느티나무 셀프BBQ_1인 수저세트',)] Epoch 150 ==============================
 Train Loss 0.06899 | Val Loss 0.36547 | SMAPE 0.71617


Training LSTM:   1%|          | 1/193 [00:30<1:37:30, 30.47s/it]


[('느티나무 셀프BBQ_1인 수저세트',)] Epoch 200 ==============================
 Train Loss 0.05668 | Val Loss 0.37487 | SMAPE 0.73919

[('느티나무 셀프BBQ_BBQ55(단체)',)] Epoch 50 ==============================
 Train Loss 0.16931 | Val Loss 0.25688 | SMAPE 0.50057

[('느티나무 셀프BBQ_BBQ55(단체)',)] Epoch 100 ==============================
 Train Loss 0.08782 | Val Loss 0.30959 | SMAPE 0.60090

[('느티나무 셀프BBQ_BBQ55(단체)',)] Epoch 150 ==============================
 Train Loss 0.05500 | Val Loss 0.30546 | SMAPE 0.57905


Training LSTM:   1%|          | 2/193 [01:01<1:37:53, 30.75s/it]

...


[('화담숲카페_아메리카노 ICE',)] Epoch 200 ==============================
 Train Loss 0.11196 | Val Loss 0.43326 | SMAPE 0.82414

[('화담숲카페_카페라떼 ICE',)] Epoch 50 ==============================
 Train Loss 0.21116 | Val Loss 0.30821 | SMAPE 0.56953

[('화담숲카페_카페라떼 ICE',)] Epoch 100 ==============================
 Train Loss 0.15286 | Val Loss 0.35072 | SMAPE 0.64308

[('화담숲카페_카페라떼 ICE',)] Epoch 150 ==============================
 Train Loss 0.10483 | Val Loss 0.35534 | SMAPE 0.65074


Training LSTM:  99%|█████████▉| 192/193 [1:37:58<00:30, 30.59s/it]


[('화담숲카페_카페라떼 ICE',)] Epoch 200 ==============================
 Train Loss 0.06501 | Val Loss 0.36554 | SMAPE 0.68005

[('화담숲카페_현미뻥스크림',)] Epoch 50 ==============================
 Train Loss 0.20693 | Val Loss 0.36448 | SMAPE 0.70820

[('화담숲카페_현미뻥스크림',)] Epoch 100 ==============================
 Train Loss 0.16410 | Val Loss 0.36835 | SMAPE 0.70345

[('화담숲카페_현미뻥스크림',)] Epoch 150 ==============================
 Train Loss 0.11104 | Val Loss 0.41156 | SMAPE 0.79342


Training LSTM: 100%|██████████| 193/193 [1:38:28<00:00, 30.62s/it]


[('화담숲카페_현미뻥스크림',)] Epoch 200 ==============================
 Train Loss 0.08389 | Val Loss 0.40751 | SMAPE 0.78084


In [ ]:
model_path = '/content/drive/MyDrive/3. Grad School/LG Aimers/Models/Trial 7/lstm_models_gpu.pkl'
lstm.save_lstm_model_gpu(trained_lstm, model_path)

In [ ]:
model_path = '/content/drive/MyDrive/3. Grad School/LG Aimers/Models/Trial 7/lstm_models_cpu.pkl'
lstm.save_lstm_model_cpu(trained_lstm, model_path)

CPU 버전 모델 저장 완료!


#### 예측하기

In [ ]:
# 저장된 모델 로드 - CPU
lookback, predict, batch_size, epochs = 28, 7, 16, 200
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model_path = '/content/drive/MyDrive/3. Grad School/LG Aimers/Models/Trial 6/lstm_models_cpu.pkl'
lstm = LSTMModel(lookback = 28, predict = 7)
trained_lstm = lstm.load_saved_model(model_path)

In [ ]:
class PredictionFunctions():
    def __init__(self, test_df = None, trained_models = None, test_prefix = None, cols = None, enc_cols = None, lookback = 28, predict = 7):
        self.test_df = test_df
        self.trained_models = trained_models
        self.test_prefix = test_prefix
        self.cols = cols
        self.enc_cols = enc_cols
        self.lookback = lookback
        self.predict = predict

    def predict_class(self, test_df, trained_models, test_prefix : str, cols : list, enc_cols : list, lookback = 28, predict = 7):
        """
        Input : test_df - test data, trained_models - list(menu : {model, encoder}), cols - x 변수들
        Output : [영업일자, 영업장명_메뉴명, 매출여부] DataFrame
        """
        results = []

        for store_menu_tup, store_test in test_df.groupby(['영업장명_메뉴명']):
            store_menu = store_menu_tup[0]
            # 훈련된 모델에 메뉴가 있는 경우만 진행
            if store_menu not in trained_models:
                continue

            # 모델 불러오기
            model = trained_models[store_menu]["model"]
            encoder = trained_models[store_menu]["encoder"]
            threshold = trained_models[store_menu].get("threshold", 0.5)

            # 변수 추가하기
            mv = Make_Variables()
            store_test['영업일자'] = pd.to_datetime(store_test['영업일자'])
            store_test_sorted = store_test.sort_values('영업일자')
            last_date = store_test_sorted['영업일자'].iloc[-1]

            future_df = mv.make_variables_test(date = last_date, test_df = store_test, predict = 7)
            encoded = encoder.transform(future_df[enc_cols])
            encoded_df = pd.DataFrame(encoded, columns = enc_cols, index = future_df.index)
            future_df[enc_cols] = encoded_df
            x = future_df[cols]

            if hasattr(model, "classes_"):
                pos_idx = int(np.where(model.classes_ == 1)[0][0])
            else:
                pos_idx = 1

            proba = model.predict_proba(x)[:, pos_idx]
            y_hat = (proba >= threshold).astype(int)


            # 예측일자: TEST_00+1일 ~ TEST_00+7일
            pred_dates = [f"{test_prefix}+{i+1}일" for i in range(predict)]

            for d, val in zip(pred_dates, y_hat):
                results.append({
                    '영업일자': d,
                    '영업장명_메뉴명': store_menu,
                    '매출여부': val
                })

        return pd.DataFrame(results)

    def predict_reg(self, test_df, trained_models, test_prefix : str, cols : list, enc_cols : list, lookback = 28, predict = 7):
        """
        Input : test_df - test data, trained_models - list(menu : {model, encoder}), cols - x 변수들
        Output : [영업일자, 영업장명_메뉴명, 매출수량] DataFrame
        """
        results = []

        for store_menu_tup, store_test in test_df.groupby(['영업장명_메뉴명']):
            store_menu = store_menu_tup[0]
            # 훈련된 모델에 메뉴가 있는 경우만 진행
            if store_menu not in trained_models:
                continue

            # 모델 불러오기
            model = trained_models[store_menu]["model"]
            encoder = trained_models[store_menu]["encoder"]

            # 변수 추가하기
            mv = Make_Variables()
            store_test['영업일자'] = pd.to_datetime(store_test['영업일자'])
            store_test_sorted = store_test.sort_values('영업일자')
            last_date = store_test_sorted['영업일자'].iloc[-1]

            future_df = mv.make_variables_test(date = last_date, test_df = store_test, predict = 7)
            encoded = encoder.transform(future_df[enc_cols])
            encoded_df = pd.DataFrame(encoded, columns = enc_cols, index = future_df.index)
            future_df[enc_cols] = encoded_df
            future_df = future_df[cols]

            # 예측일자: TEST_00+1일 ~ TEST_00+7일
            pred_dates = [f"{test_prefix}+{i+1}일" for i in range(predict)]

            for d, val in zip(pred_dates, model.predict(future_df)):
                results.append({
                    '영업일자': d,
                    '영업장명_메뉴명': store_menu,
                    '매출수량': val
                })

        return pd.DataFrame(results)

    def predict_lstm(self, test_df, trained_models, test_prefix : str, cols : list, enc_cols : list, num_cols : list, lookback = 28, predict = 7, device = None):
        """
        Input : test_df - test data, trained_models - list(menu : { model}), cols - x 변수들
        Output : [영업일자, 영업장명_메뉴명, 매출수량] DataFrame
        """

        if device is None:
            device = getattr(self, "device", "cpu")
        device = torch.device(device)

        results = []

        # 매장, 메뉴별로 그룹화해서 예측
        for store_menu, store_test in test_df.groupby(['영업장명_메뉴명']):
            # 훈련된 모델에 메뉴가 있는 경우만 진행
            if store_menu not in trained_models:
                continue

            # 모델, scaler 불러오기
            state = trained_models[store_menu]['model']
            meta = trained_models[store_menu]['meta']
            encoders = trained_models[store_menu]['encoders']
            features_scaler = trained_models[store_menu]['features_scaler']
            target_scaler = trained_models[store_menu]['target_scaler']

            # LSTM 모델 생성
            model = MultiOutputLSTM(
                input_dim=meta['input_dim'],
                hidden_dim=meta['hidden_dim'],
                num_layers=meta['num_layers'],
                output_dim=meta['output_dim']
            )

            model.load_state_dict(state)
            model = model.to(device).eval()

            # LSTM 입력으로 활용할 최근 lookback 만큼의 데이터 가져오기
            mv = Make_Variables()
            store_test = store_test.sort_values('영업일자')
            store_test = mv.make_variables_train(data = store_test)
            store_test_sorted = store_test.sort_values('영업일자')

            features = cols + ["매출수량"]
            if len(store_test_sorted) < lookback:
                continue

            recent_df = store_test_sorted[features].iloc[-lookback:].copy()
            if len(recent_df) < lookback:
                continue # lookback 만큼의 데이터가 없으면 예측 안 하고 넘어가기

            ##### 요기서 변수 추가
            last_date = store_test_sorted['영업일자'].iloc[-1]
            recent_df_for_mv = store_test_sorted[features + ['영업장명_메뉴명', '영업일자']].iloc[-lookback:].copy()
            future_df = mv.make_variables_test(date = last_date, test_df = recent_df_for_mv, predict = 7)
            future_df['매출수량'] = 0.0
            full_df = pd.concat([recent_df, future_df[features]], axis = 0)

            # enc_cols 정규화
            for col in enc_cols:
                if col not in full_df.columns:
                    continue
                if col in encoders:
                    le = encoders[col]
                    full_df[col] = le.transform(full_df[col].astype(str)) ##### str로 바꿔봄...
                else:
                    full_df[col] = full_df[col].astype(int)

            # num_cols 스케일링
            full_df[num_cols] = features_scaler.transform(full_df[num_cols])

            x_input_vals = full_df[cols].values
            x_input = x_input_vals[:lookback]
            x_input = torch.tensor([x_input]).float().to(device)

            # 예측 수행
            with torch.no_grad():
                pred_scaled = model(x_input).squeeze().cpu().numpy()

            # 역정규화
            restored = []
            for i in range(predict):
                dummy = np.zeros((1, len(features)))
                dummy[0, features.index("매출수량")] = pred_scaled[i]
                restored_val = target_scaler.inverse_transform(dummy)[0, features.index("매출수량")]
                restored.append(max(restored_val, 0)) # 음수 나오면 0으로 처리

            # 예측일자: TEST_00+1일 ~ TEST_00+7일
            pred_dates = [f"{test_prefix}+{i+1}일" for i in range(predict)]

            for d, val in zip(pred_dates, restored):
                results.append({
                    '영업일자': d,
                    '영업장명_메뉴명': store_menu,
                    '매출수량(lstm)': val
                })

        return pd.DataFrame(results)

In [ ]:
    def predict_reg_model(self, test_df, trained_model, test_prefix : str, cols : list, enc_cols : list, data, lookback = 28, predict = 7):
        """
        Input : test_df - test data, trained_model - {~~}, original_data : train할 때 사용한 데이터 (data)
        Output : [영업일자, 영업장명_메뉴명, 매출수량] DataFrame
        """
        results = []

        # 모델 불러오기
        model = trained_model["model"]
        encoder = trained_model["encoder"]

        # 변수 추가하기
        mv = Make_Variables()

        for store_menu, store_df in test_df.groupby(['영업장명_메뉴명']):

            store_df['영업일자'] = pd.to_datetime(store_df['영업일자'])
            store_df_sorted = store_df.sort_values('영업일자')
            last_date = store_df_sorted['영업일자'].iloc[-1]

            future_df = mv.make_variables_test(date = last_date, test_df = store_df, original_data = data, predict = 7)
            if enc_cols:
                future_df[enc_cols] = encoder.transform(future_df[enc_cols])

            # 사용할 변수만
            future_df = future_df[cols]

            # 로그 변환 처리
            predicted = model.predict(future_df)

            # 예측일자: TEST_00+1일 ~ TEST_00+7일
            pred_dates = [f"{test_prefix}+{i+1}일" for i in range(predict)]

            for d, val in zip(pred_dates, predicted):
                results.append({
                    '영업일자': d,
                    '영업장명_메뉴명': store_menu,
                    '매출수량': val
                })

        return pd.DataFrame(results)

#### 예측값 생성

In [ ]:
import re
all_preds_class = []
all_preds_reg = []
all_preds_lstm = []

# 모든 test_*.csv 순회
test_files = sorted(glob.glob('DATA/test/TEST_*.csv'))
predictions = PredictionFunctions()

for path in test_files:
    test_df = pd.read_csv(path)

    # 파일명에서 접두어 추출 (예: TEST_00)
    filename = os.path.basename(path)
    test_prefix = re.search(r'(TEST_\d+)', filename).group(1)

    # lstm 넣고
    pred_lstm = predictions.predict_lstm(test_df, trained_lstm, test_prefix, cols, enc_cols, num_cols, device = 'cpu')
    all_preds_lstm.append(pred_lstm)

df_lstm  = pd.concat(all_preds_lstm, ignore_index=True)

         영업일자            영업장명_메뉴명  매출수량
0  2024-06-16  느티나무 셀프BBQ_1인 수저세트     2
1  2024-06-17  느티나무 셀프BBQ_1인 수저세트     0
2  2024-06-18  느티나무 셀프BBQ_1인 수저세트     0
3  2024-06-19  느티나무 셀프BBQ_1인 수저세트     0
4  2024-06-20  느티나무 셀프BBQ_1인 수저세트     4
          영업일자              영업장명_메뉴명  매출수량
28  2024-06-16  느티나무 셀프BBQ_BBQ55(단체)     0
29  2024-06-17  느티나무 셀프BBQ_BBQ55(단체)     0
30  2024-06-18  느티나무 셀프BBQ_BBQ55(단체)     0
31  2024-06-19  느티나무 셀프BBQ_BBQ55(단체)    67
32  2024-06-20  느티나무 셀프BBQ_BBQ55(단체)   124
          영업일자                영업장명_메뉴명  매출수량
56  2024-06-16  느티나무 셀프BBQ_대여료 30,000원     5
57  2024-06-17  느티나무 셀프BBQ_대여료 30,000원     0
58  2024-06-18  느티나무 셀프BBQ_대여료 30,000원     2
59  2024-06-19  느티나무 셀프BBQ_대여료 30,000원     1
60  2024-06-20  느티나무 셀프BBQ_대여료 30,000원     0
          영업일자                영업장명_메뉴명  매출수량
84  2024-06-16  느티나무 셀프BBQ_대여료 60,000원     3
85  2024-06-17  느티나무 셀프BBQ_대여료 60,000원     0
86  2024-06-18  느티나무 셀프BBQ_대여료 60,000원     0
87  2024-06-19  느티나무 셀프BBQ_대여료 60,000원     0
88  202

KeyboardInterrupt: 

In [ ]:
len(df_lstm)

13510

In [ ]:
import re
all_preds_class = []
all_preds_reg = []
all_preds_lstm = []


# 모든 test_*.csv 순회
test_files = sorted(glob.glob('DATA/test/TEST_*.csv'))
predictions = PredictionFunctions()

for path in test_files:
    test_df = pd.read_csv(path)

    # 파일명에서 접두어 추출 (예: TEST_00)
    filename = os.path.basename(path)
    test_prefix = re.search(r'(TEST_\d+)', filename).group(1)

    # 일단 분류 모델 넣고
    pred_class = predictions.predict_class(test_df, models_class, test_prefix, cols, enc_cols)
    all_preds_class.append(pred_class)

    # 1 나오면 회귀 모델 넣고
    pred_reg = predictions.predict_reg(test_df, models_reg, test_prefix, cols, enc_cols)
    all_preds_reg.append(pred_reg)

    # lstm 넣고
    pred_lstm = predictions.predict_lstm(test_df, trained_lstm, test_prefix, cols, enc_cols, num_cols)
    all_preds_lstm.append(pred_lstm)

    print("예측 완료 !")

    # 합치기 (가중치.. 일단은 1.5 / 8.5 정도....)

df_class = pd.concat(all_preds_class, ignore_index = True)
df_reg   = pd.concat(all_preds_reg, ignore_index = True)
df_lstm  = pd.concat(all_preds_lstm, ignore_index = True)

예측 완료 !
예측 완료 !
예측 완료 !
예측 완료 !
예측 완료 !
예측 완료 !
예측 완료 !
예측 완료 !
예측 완료 !
예측 완료 !


In [ ]:
full_pred_df = pd.merge(df_class, df_reg, on=['영업일자', '영업장명_메뉴명'], how='outer')
full_pred_df.rename(columns={'매출수량': '매출수량(reg)'}, inplace=True)

df_lstm_plz = df_lstm.copy()
df_lstm_plz['영업장명_메뉴명'] = df_lstm['영업장명_메뉴명'].apply(lambda x: x[0] if isinstance(x, tuple) else x)

full_pred_df = pd.merge(df_lstm_plz, full_pred_df, on=['영업일자', '영업장명_메뉴명'], how='outer')
full_pred_df.head()

,영업일자,영업장명_메뉴명,매출수량(lstm),매출여부,매출수량(reg)
0,TEST_00+1일,느티나무 셀프BBQ_1인 수저세트,8.494163,1,10.307570
1,TEST_00+1일,느티나무 셀프BBQ_BBQ55(단체),57.814996,0,75.361053
2,TEST_00+1일,"느티나무 셀프BBQ_대여료 30,000원",7.369763,1,10.134176
3,TEST_00+1일,"느티나무 셀프BBQ_대여료 60,000원",1.446719,1,1.488075
4,TEST_00+1일,"느티나무 셀프BBQ_대여료 90,000원",0.937538,1,1.748883


In [ ]:
full_pred_df['매출수량'] = np.where(
    full_pred_df['매출여부'] == 1,
    full_pred_df['매출수량(reg)'] * 0.3 + full_pred_df['매출수량(lstm)'] * 0.7,
    full_pred_df['매출수량(lstm)']
)

full_pred_df.drop(columns=['매출여부', '매출수량(reg)', '매출수량(lstm)'], inplace=True)
full_pred_df

,영업일자,영업장명_메뉴명,매출수량
0,TEST_00+1일,느티나무 셀프BBQ_1인 수저세트,9.038186
1,TEST_00+1일,느티나무 셀프BBQ_BBQ55(단체),57.814996
2,TEST_00+1일,"느티나무 셀프BBQ_대여료 30,000원",8.199087
3,TEST_00+1일,"느티나무 셀프BBQ_대여료 60,000원",1.459126
4,TEST_00+1일,"느티나무 셀프BBQ_대여료 90,000원",1.180942
...,...,...,...
13505,TEST_09+7일,화담숲카페_메밀미숫가루,52.605764
13506,TEST_09+7일,화담숲카페_아메리카노 HOT,33.743139
13507,TEST_09+7일,화담숲카페_아메리카노 ICE,101.176566
13508,TEST_09+7일,화담숲카페_카페라떼 ICE,22.974551


In [ ]:
full_pred_df['매출수량'] = np.where(
    full_pred_df['매출여부'] == 1,
    full_pred_df['매출수량(reg)'] * 0.4 + full_pred_df['매출수량(lstm)'] * 0.6,
    full_pred_df['매출수량(lstm)']
)

full_pred_df.drop(columns=['매출여부', '매출수량(reg)', '매출수량(lstm)'], inplace=True)
full_pred_df

,영업일자,영업장명_메뉴명,매출수량
0,TEST_00+1일,느티나무 셀프BBQ_1인 수저세트,7.233946
1,TEST_00+1일,느티나무 셀프BBQ_BBQ55(단체),75.649501
2,TEST_00+1일,"느티나무 셀프BBQ_대여료 30,000원",8.881719
3,TEST_00+1일,"느티나무 셀프BBQ_대여료 60,000원",1.487016
4,TEST_00+1일,"느티나무 셀프BBQ_대여료 90,000원",1.289370
...,...,...,...
13505,TEST_09+7일,화담숲카페_메밀미숫가루,49.772997
13506,TEST_09+7일,화담숲카페_아메리카노 HOT,27.679980
13507,TEST_09+7일,화담숲카페_아메리카노 ICE,92.439979
13508,TEST_09+7일,화담숲카페_카페라떼 ICE,25.499672


In [ ]:
def convert_to_submission_format(pred_df: pd.DataFrame, sample_submission: pd.DataFrame):
    # (영업일자, 메뉴) → 매출수량 딕셔너리로 변환
    pred_dict = dict(zip(
        zip(pred_df['영업일자'], pred_df['영업장명_메뉴명']),
        pred_df['매출수량'].astype(float)
    ))

    final_df = sample_submission.copy()

    menu_cols = final_df.columns[1:]
    final_df[menu_cols] = final_df[menu_cols].astype(float)

    for row_idx in final_df.index:
        date = final_df.loc[row_idx, '영업일자']
        for col in final_df.columns[1:]:  # 메뉴명들
            final_df.loc[row_idx, col] = pred_dict.get((date, col), 0)

    return final_df

In [ ]:
full_pred_df_notzero = full_pred_df.copy()
full_pred_df_notzero.loc[full_pred_df_notzero['매출수량'].abs() < 1e-9, '매출수량'] = 1

sample_submission = pd.read_csv('DATA/sample_submission.csv')
final_hybrid = convert_to_submission_format(full_pred_df, sample_submission)
final_hybrid.to_csv('baseline_submission_hybrid.csv', index=False, encoding='utf-8-sig')

In [ ]:
df_lstm_plz = df_lstm.copy()
df_lstm_plz['영업장명_메뉴명'] = df_lstm['영업장명_메뉴명'].apply(lambda x: x[0] if isinstance(x, tuple) else x)
df_lstm_end = df_lstm_plz.rename(columns = {'매출수량(lstm)' : '매출수량'})

# 1 변환
df_lstm_end.loc[df_lstm_end['매출수량'].abs() < 1e-9, '매출수량'] = 1

sample_submission = pd.read_csv('DATA/sample_submission.csv')
final_lstm = convert_to_submission_format(df_lstm_end, sample_submission)
final_lstm.to_csv('baseline_submission_lstm.csv', index=False, encoding='utf-8-sig')

In [ ]:
df_reg_notzero = df_reg.copy()
df_reg_notzero.loc[df_reg_notzero['매출수량'].abs() < 1e-9, '매출수량'] = 1

sample_submission = pd.read_csv('DATA/sample_submission.csv')
final_hybrid = convert_to_submission_format(df_reg, sample_submission)
final_hybrid.to_csv('baseline_submission_reg.csv', index=False, encoding='utf-8-sig')

#### XAI

##### LSTM 모델

In [ ]:
# 저장된 모델 로드 - CPU
lookback, predict, batch_size, epochs = 28, 7, 16, 200
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model_path = '/content/drive/MyDrive/3. Grad School/LG Aimers/Models/Trial 6/lstm_models_cpu.pkl'
lstm = LSTMModel(lookback = 28, predict = 7)
trained_lstm = lstm.load_saved_model(model_path)

In [ ]:
# ! pip install captum
from captum.attr import IntegratedGradients

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 514.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18

In [ ]:
import numpy as np
import pandas as pd
import torch
from captum.attr import IntegratedGradients
from collections import defaultdict
from typing import Dict, List

In [ ]:
import numpy as np
import pandas as pd
from typing import Dict, List, Optional

def build_input_last_window(
    df: pd.DataFrame,
    menu_name: str,
    mv,  # Make_Variables() 인스턴스
    cols: List[str],
    enc_cols: List[str],
    num_cols: List[str],
    encoders: Dict[str, object],
    features_scaler,
    lookback: int,
    debug: bool = False,
) -> Optional[np.ndarray]:
    """
    해당 메뉴 df에서 마지막 lookback 윈도우(1개)를 만들어 (lookback, F) numpy 반환.
    예측 시와 동일한 전처리를 최대한 재현. (없거나 부족하면 None 반환)

    Params
    ------
    df : 전체 데이터프레임 (여러 메뉴 포함)
    menu_name : trained_models의 키(문자열 or ('키',) 형태 튜플일 수 있음)
    mv : Make_Variables 인스턴스 (make_variables_train 사용)
    cols : 최종 입력 피처 컬럼 리스트 (모델 학습 시 사용한 순서)
    enc_cols : 인코딩할 범주형 컬럼 리스트
    num_cols : 스케일링할 수치형 컬럼 리스트
    encoders : {col: fitted LabelEncoder-like} 딕셔너리
    features_scaler : fitted scaler (예: StandardScaler/MinMaxScaler)
    lookback : LSTM lookback 길이
    debug : True면 중간 정보 프린트

    Returns
    -------
    np.ndarray shape (lookback, len(cols)) or None
    """

    # 1) 메뉴 키를 문자열로 정규화
    key = menu_name
    if isinstance(key, (tuple, list)):
        key = key[0]
    key = str(key)

    # 2) df 쪽도 문자열 비교로 통일
    df_key = df['영업장명_메뉴명'].astype(str)
    store = df[df_key == key].copy()
    if store.empty:
        if debug:
            print(f"[build_input_last_window] menu='{key}' → store.empty=True")
        return None

    # 정렬 (필수)
    store = store.sort_values('영업일자')

    # 3) 파생변수 생성 (함수 내부에서 NaN 생기지 않게 주의)
    store = mv.make_variables_train(store)
    store = store.sort_values('영업일자').reset_index(drop=True)

    if len(store) < lookback:
        if debug:
            print(f"[build_input_last_window] menu='{key}' → len(store)={len(store)} < lookback={lookback}")
        return None

    # 4) 입력 특징 행렬 (lookback 길이만큼 슬라이스)
    #    필요한 컬럼이 전부 있는지 먼저 체크(없으면 없는 것만 제외하고 경고)
    missing_cols = [c for c in cols if c not in store.columns]
    if missing_cols and debug:
        print(f"[build_input_last_window][WARN] menu='{key}' missing cols: {missing_cols}")

    # 존재하는 컬럼만 사용 (모델과 cols가 반드시 일치해야 한다면 여기서 None 반환해도 됨)
    use_cols = [c for c in cols if c in store.columns]
    if len(use_cols) == 0:
        if debug:
            print(f"[build_input_last_window][ERROR] menu='{key}' no usable cols in `cols`.")
        return None

    full = store[use_cols].iloc[-lookback:].copy()

    # 5) enc_cols 인코딩 (학습 시 encoders가 있다면 transform 사용)
    for c in enc_cols:
        if c not in full.columns:
            continue
        s = full[c]
        if c in encoders:
            le = encoders[c]
            # 문자열로 통일 후 transform
            try:
                full[c] = le.transform(s.astype(str))
            except Exception as e:
                # 학습 시 없던 새로운 레이블 등장 시 대응(필요하면 더 정교하게 처리)
                if debug:
                    print(f"[build_input_last_window][ENC WARN] menu='{key}' col='{c}' transform error: {e}. Fallback to factorize.")
                full[c] = pd.factorize(s.astype(str))[0]
        else:
            # 수치형이 아니면 안전하게 factorize
            if not np.issubdtype(full[c].dtype, np.number):
                full[c] = pd.factorize(s.astype(str))[0]

    # 6) num_cols 스케일링 (교집합만 적용)
    if num_cols:
        scale_cols = [c for c in num_cols if c in full.columns]
        if scale_cols:
            full[scale_cols] = features_scaler.transform(full[scale_cols])

    # 7) 결측 안전 처리
    full = full.fillna(0.0)

    # 8) 최종 컬럼 순서를 원래 `cols` 순서로 맞춰주되, 없는 건 제외
    #    (모델 입력 차원 순서를 맞추기 위함)
    final_cols = [c for c in cols if c in full.columns]
    X = full[final_cols].to_numpy(dtype=np.float32)

    # 만약 모델이 정확히 len(cols) 차원을 기대한다면, 여기서 길이 검증
    if len(final_cols) != len(cols):
        if debug:
            missing_for_model = [c for c in cols if c not in final_cols]
            print(f"[build_input_last_window][WARN] menu='{key}' some model cols missing in final input: {missing_for_model}")

    # shape: (lookback, F_used)
    return X



def aggregate_feature_importance_IG(
    test_df: pd.DataFrame,
    trained_models: Dict[str, dict],
    mv,  # Make_Variables() 인스턴스
    MultiOutputLSTM,  # 모델 클래스
    cols: List[str],
    enc_cols: List[str],
    num_cols: List[str],
    lookback: int,
    predict: int,
    device: str = "cpu",
    ig_steps: int = 64,          # IG 적분 스텝
    baseline_mode: str = "zero", # "zero" or "mean"
):
    """
    모든 메뉴(모델)에 대해:
      1) 마지막 윈도우 입력 x (1, T, F)를 만들고
      2) 각 호라이즌 k에 대해 IG(Integrated Gradients)로 (T,F) 기여도 계산
      3) 절대값 합으로 특성별 중요도 집계
    반환:
      - overall_df: 전체 메뉴×호라이즌 집계한 최종 특성 중요도
      - per_horizon_df: 호라이즌별 특성 중요도 (각 k 행)
    """
    device = torch.device(device)

    # 전역 집계 버퍼
    feat_total = np.zeros(len(cols), dtype=np.float64)               # 전체 특성 중요도
    feat_total_by_h = np.zeros((predict, len(cols)), dtype=np.float64)  # 호라이즌별 특성 중요도

    # baseline 준비용 mean 벡터 (선택)
    mean_input = None
    if baseline_mode == "mean":
        # 간단히: 모든 메뉴의 마지막 윈도우를 모아 평균 (메모리 아끼려면 일부만 샘플)
        stacks = []
        for menu, bundle in trained_models.items():
            x_np = build_input_last_window(
                test_df, menu, mv, cols, enc_cols, num_cols,
                bundle['encoders'], bundle['features_scaler'], lookback
            )
            if x_np is not None:
                stacks.append(x_np)
        if len(stacks) == 0:
            raise ValueError("평균 baseline을 만들 입력이 없습니다.")
        mean_input = np.mean(np.stack(stacks, axis=0), axis=0)  # (T,F)

    for menu, bundle in trained_models.items():
        state = bundle['model']
        meta  = bundle['meta']
        encoders = bundle['encoders']
        fscaler  = bundle['features_scaler']

        # 모델 로드
        model = MultiOutputLSTM(
            input_dim=meta['input_dim'],
            hidden_dim=meta['hidden_dim'],
            num_layers=meta['num_layers'],
            output_dim=meta['output_dim']
        ).to(device).eval()
        model.load_state_dict(state)

        # 입력 만들기
        x_np = build_input_last_window(
            test_df, menu, mv, cols, enc_cols, num_cols, encoders, fscaler, lookback
        )
        if x_np is None:  # 데이터 부족
            continue

        x = torch.from_numpy(x_np).unsqueeze(0).to(device)  # (1,T,F)
        x = x.detach()
        x.requires_grad_(True)

        # baseline
        if baseline_mode == "zero":
            baseline = torch.zeros_like(x)
        else:
            baseline = torch.from_numpy(mean_input).unsqueeze(0).to(device)

        # 호라이즌별로 IG 계산
        for day_k in range(predict):
            def forward_for_k(inp):
                out = model(inp)          # (N, predict)
                return out[:, day_k]      # (N,)

            ig = IntegratedGradients(forward_for_k)
            attr, _ = ig.attribute(
                inputs=x, baselines=baseline, n_steps=ig_steps, return_convergence_delta=True
            )  # (1,T,F)

            A = attr.squeeze(0).detach().cpu().numpy()  # (T,F)

            # 시간축으로 합산(또는 평균)하여 특성 중요도 벡터로
            feat_imp_k = np.abs(A).sum(axis=0)  # (F,)
            feat_total_by_h[day_k] += feat_imp_k
            feat_total += feat_imp_k

    # 정리: pandas로 출력
    overall_df = pd.DataFrame({
        "feature": cols,
        "importance": feat_total
    }).sort_values("importance", ascending=False).reset_index(drop=True)

    per_horizon_df = pd.DataFrame(
        feat_total_by_h, columns=cols
    )
    per_horizon_df.index = [f"H+{k+1}" for k in range(predict)]

    # 정규화(선택): 총합=1
    if overall_df["importance"].sum() > 0:
        overall_df["norm_importance"] = overall_df["importance"] / overall_df["importance"].sum()
    per_horizon_norm = per_horizon_df.div(per_horizon_df.sum(axis=1).replace(0, np.nan), axis=0)

    return overall_df, per_horizon_df, per_horizon_norm

In [ ]:
# 모든 test_*.csv 순회
test_files = sorted(glob.glob('DATA/test/TEST_*.csv'))

for path in test_files[:1]:
    test_df = pd.read_csv(path)

In [ ]:
# 준비물 예시
mv = Make_Variables()
device = "cuda" if torch.cuda.is_available() else "cpu"

overall_df, per_horizon_df, per_horizon_norm = aggregate_feature_importance_IG(
    test_df=test_df,
    trained_models=trained_lstm,         # 당신의 dict
    mv=mv,
    MultiOutputLSTM=MultiOutputLSTM,     # 당신의 클래스
    cols=cols,
    enc_cols=enc_cols,
    num_cols=num_cols,
    lookback=28,
    predict=7,
    device=device,
    ig_steps=64,
    baseline_mode="zero",   # 또는 "mean"
)

print(overall_df)      # 전반적 변수 중요도 Top20
print(per_horizon_norm)  # 호라이즌별 중요도 (행정규화)

                feature  importance  norm_importance
0                season   84.869912         0.206079
1             month_cos   44.624976         0.108357
2           weekday_cos   41.015457         0.099593
3           weekday_sin   36.113747         0.087690
4               day_sin   32.110272         0.077969
5               day_cos   20.873418         0.050684
6   is_holiday_sandwich   19.185584         0.046586
7             month_sin   19.170896         0.046550
8            prev_sd_21   13.856936         0.033647
9            prev_sd_14   13.843205         0.033614
10            prev_sd_7   13.391129         0.032516
11          prev_avg_21   12.042355         0.029241
12           prev_avg_7   11.037719         0.026801
13          prev_avg_14   10.936975         0.026557
14             year_enc   10.776368         0.026167
15           is_holiday    9.935498         0.024125
16         month_weight    7.548724         0.018330
17          is_sandwich    6.332801         0.

In [ ]:
overall_df

,feature,importance,norm_importance
0,season,84.869912,0.206079
1,month_cos,44.624976,0.108357
2,weekday_cos,41.015457,0.099593
3,weekday_sin,36.113747,0.087690
4,day_sin,32.110272,0.077969
5,day_cos,20.873418,0.050684
6,is_holiday_sandwich,19.185584,0.046586
7,month_sin,19.170896,0.046550
8,prev_sd_21,13.856936,0.033647
9,prev_sd_14,13.843205,0.033614


In [ ]:
per_horizon_df

,year_enc,month_sin,month_cos,day_sin,day_cos,weekday_sin,weekday_cos,season,is_holiday,is_sandwich,is_holiday_sandwich,month_weight,week_weight,prev_avg_7,prev_avg_14,prev_avg_21,prev_sd_7,prev_sd_14,prev_sd_21
H+1,1.604334,2.790146,6.772316,5.037103,3.575934,5.555296,6.240763,12.765112,1.729429,1.072913,3.100922,1.270241,0.634946,1.669149,1.548195,1.708743,2.009152,2.006677,2.020215
H+2,1.659375,2.602959,6.719466,4.541760,3.386222,5.239784,5.877996,12.382138,1.570435,0.978376,2.959194,1.199221,0.585311,1.550529,1.485531,1.705968,1.907065,1.960410,1.961275
H+3,1.391010,2.401211,5.489500,3.933792,2.593494,4.382369,5.026776,10.892910,1.285235,0.812130,2.450873,0.965381,0.496349,1.303348,1.337872,1.469338,1.615817,1.694023,1.688270
H+4,1.359214,2.503329,5.415160,3.940078,2.393078,4.159656,4.649391,10.661720,1.112991,0.769818,2.333372,0.923385,0.472400,1.344457,1.386320,1.518058,1.598106,1.702183,1.690636
H+5,1.509117,2.702943,6.122737,4.513648,2.639076,4.883200,5.314176,11.709555,1.153136,0.790762,2.452519,0.973754,0.586854,1.518178,1.632551,1.783480,1.871783,1.979664,1.943044
H+6,1.611569,2.998806,6.945364,4.791700,3.023415,5.759425,6.664727,13.137104,1.401579,0.916469,2.740308,1.077440,0.645174,1.797734,1.780360,1.932157,2.096748,2.188317,2.230893
H+7,1.641750,3.171501,7.160433,5.352190,3.262199,6.134017,7.241629,13.321374,1.682693,0.992333,3.148396,1.139302,0.745629,1.854323,1.766147,1.924612,2.292459,2.311930,2.322603
